# Gene Annotation (Best transcripts)

## Best-transcripts tracks for hg19 

In https://genome.ucsc.edu/FAQ/FAQgenes.html#gene we can read:

The following gene tracks have "best-transcripts" tracks: For hg19 (UCSC Genes on hg19), the knownCanonical table is a subset of the UCSC Genes track. It was generated at UCSC by identifying a canonical isoform for each cluster ID, or gene. Generally, this is the longest isoform. It can be downloaded directly from the hg19 downloads database or by using the Table Browser.  
https://genome.ucsc.edu/cgi-bin/hgTrackUi?db=hg19&g=knownGene

Data: https://hgdownload.soe.ucsc.edu/goldenPath/hg19/database/knownCanonical.txt.gz

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Read dataset
file_in = 'knownCanonical.txt.gz'
df_best_transcripts = pd.read_csv(file_in, sep='\t', header=None, comment='#')
# chrom	        chrX	            varchar(255)	values	Chromosome
# chromStart	100627107	        int(11)	        range	Start position (0 based). Represents transcription start for + strand genes, end for - strand genes
# chromEnd	    100636806	        int(11)	        range	End position (non-inclusive). Represents transcription end for + strand genes, start for - strand genes
# clusterId	    1	                int(11)	        range	Which cluster of transcripts this belongs to in knownIsoforms
# transcript	ENST00000373020.9	varchar(255)	values	Corresponds to knownGene name field.
# protein	    ENSG00000000003.15	varchar(255)	values	Accession of the associated protein, or UCSC ID in newer tables.
df_best_transcripts.columns = ['Chromosome', 'Start_position', 'End_position', 'cluster', 'name', 'UCSC_id']
df_best_transcripts.head(5)

,Chromosome,Start_position,End_position,cluster,name,UCSC_id
0,chr1,11873,14409,1,uc010nxq.1,uc010nxq.1
1,chr1,14361,19759,2,uc009viu.3,uc009viu.3
2,chr1,14406,29370,3,uc009viw.2,uc009viw.2
3,chr1,34610,36081,4,uc001aak.3,uc001aak.3
4,chr1,69090,70008,5,uc001aal.1,uc001aal.1


In [3]:
# Remove columns not needed
columns_to_remove = ['Chromosome', 'Start_position', 'End_position', 'cluster', 'UCSC_id']
df_best_transcripts.drop(columns_to_remove, axis=1, inplace=True)
print("Size: ", df_best_transcripts.shape)
df_best_transcripts.head(5)

Size:  (31848, 1)


,name
0,uc010nxq.1
1,uc009viu.3
2,uc009viw.2
3,uc001aak.3
4,uc001aal.1


## Finding the equivalence between UCSC and GENCODE/Ensembl format

Equivalences table UCSC-vs-GENCODE: https://hgdownload.soe.ucsc.edu/goldenPath/hg19/database/knownToEnsembl.txt.gz

Coming from directory: https://hgdownload.soe.ucsc.edu/goldenPath/hg19/database/

In [4]:
file_in = 'knownToEnsembl.txt.gz'
df_equivalences = pd.read_csv(file_in, sep='\t', header=None, comment='#')
df_equivalences.columns = ['name', 'transcript']
print("Size: ", df_equivalences.shape)
df_equivalences.head(5)

Size:  (76178, 2)


,name,transcript
0,uc022bqo.2,ENST00000389680
1,uc004cor.1,ENST00000387342
2,uc004cos.5,ENST00000387347
3,uc031tga.1,ENST00000361624
4,uc011mfi.2,ENST00000361739


## Merge both dataframes

In [5]:
df_best_transcripts = pd.merge(df_best_transcripts, df_equivalences, how="inner", on="name")
print("Size: ", df_best_transcripts.shape)
df_best_transcripts.head(5)

Size:  (26811, 2)


,name,transcript
0,uc010nxq.1,ENST00000518655
1,uc009viu.3,ENST00000438504
2,uc009viw.2,ENST00000438504
3,uc001aak.3,ENST00000417324
4,uc001aal.1,ENST00000335137


## Add Length

In [6]:
# Read GFF3 (we have to take length from GFF3 file, where exons have been taken into account)
file='GFF3.csv'
df_GFF3 = pd.read_csv(file,sep=';',na_values='n.a.',index_col=0)
print("Size: ", df_GFF3.shape)
df_GFF3.head(5)

Size:  (196520, 2)


,gene,Length
ENST00000456328,ENSG00000223972,1657
ENST00000515242,ENSG00000223972,1653
ENST00000518655,ENSG00000223972,1483
ENST00000450305,ENSG00000223972,632
ENST00000438504,ENSG00000227232,1783


In [7]:
df_best_transcripts = pd.merge(df_best_transcripts, df_GFF3, how="inner", left_on="transcript", right_index=True)

print("Size: ", df_best_transcripts.shape)
df_best_transcripts.head(5)

Size:  (25637, 4)


,name,transcript,gene,Length
0,uc010nxq.1,ENST00000518655,ENSG00000223972,1483
1,uc009viu.3,ENST00000438504,ENSG00000227232,1783
2,uc009viw.2,ENST00000438504,ENSG00000227232,1783
3,uc001aak.3,ENST00000417324,ENSG00000237613,1187
4,uc001aal.1,ENST00000335137,ENSG00000186092,918


## Check how many transcripts per gene (should be only 1)

In [9]:
df_best_transcripts_per_gene = df_best_transcripts.groupby("gene").count()
df_best_transcripts_per_gene = df_best_transcripts_per_gene.rename(columns = {'name':'number_of_transcripts'})
df_best_transcripts_per_gene.drop(['transcript','Length'], axis = 1, inplace = True)
df_best_transcripts_per_gene.describe()

,number_of_transcripts
count,25025.000000
mean,1.024456
std,0.183763
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,13.000000


In [10]:
df_best_transcripts_per_gene.loc[df_best_transcripts_per_gene['number_of_transcripts']>1]

,number_of_transcripts
gene,
ENSG00000002834,2
ENSG00000006327,2
ENSG00000007047,2
ENSG00000007392,2
ENSG00000011275,2
...,...
ENSG00000269609,2
ENSG00000269834,2
ENSG00000269893,2


In [11]:
# Let's analyze a couple of them
gene_name = 'ENSG00000002834'
df_best_transcripts.loc[df_best_transcripts['gene'] == gene_name]

,name,transcript,gene,Length
10384,uc010cvq.3,ENST00000318008,ENSG00000002834,4109
10385,uc002hra.3,ENST00000318008,ENSG00000002834,4109


This one has the same information. We can remove one of the rows

In [12]:
gene_name = 'ENSG00000006327'
df_best_transcripts.loc[df_best_transcripts['gene'] == gene_name]

,name,transcript,gene,Length
8830,uc002csv.4,ENST00000326577,ENSG00000006327,1033
8831,uc031quu.1,ENST00000571351,ENSG00000006327,1678


This one has two different transcripts and lengths

In [13]:
# Remove those cases that have same transcript name, gene name and length
df_best_transcripts.drop_duplicates(subset=['transcript', 'gene', 'Length'], keep='first', inplace=True)
print("Size: ", df_best_transcripts.shape)
df_best_transcripts.head()

Size:  (25508, 4)


,name,transcript,gene,Length
0,uc010nxq.1,ENST00000518655,ENSG00000223972,1483
1,uc009viu.3,ENST00000438504,ENSG00000227232,1783
3,uc001aak.3,ENST00000417324,ENSG00000237613,1187
4,uc001aal.1,ENST00000335137,ENSG00000186092,918
5,uc021oeg.2,ENST00000493797,ENSG00000239906,323


In [14]:
df_best_transcripts_per_gene = df_best_transcripts.groupby("gene").count()
df_best_transcripts_per_gene = df_best_transcripts_per_gene.rename(columns = {'name':'number_of_transcripts'})
df_best_transcripts_per_gene.loc[df_best_transcripts_per_gene['number_of_transcripts']>1]

,number_of_transcripts,transcript,Length
gene,,,
ENSG00000006327,2,2,2
ENSG00000007047,2,2,2
ENSG00000007392,2,2,2
ENSG00000011275,2,2,2
ENSG00000011347,2,2,2
...,...,...,...
ENSG00000269501,2,2,2
ENSG00000269609,2,2,2
ENSG00000269834,2,2,2


In [15]:
# Check how many genes have more than 1 transcript
print("Number of genes: ", df_best_transcripts_per_gene.shape)
print("Number of genes with more than 1 trasncript: ", df_best_transcripts_per_gene.loc[df_best_transcripts_per_gene['number_of_transcripts']>1].shape)
print("Percentage: ", (456/25025)*100, "%")

Number of genes:  (25025, 3)
Number of genes with more than 1 trasncript:  (456, 3)
Percentage:  1.8221778221778222 %


## Save dataframe

In [16]:
# Save file
file='BestTranscript.csv'
df_best_transcripts.to_csv(file, sep=';', index = False)

In [17]:
df_best_transcripts.head(3)

,name,transcript,gene,Length
0,uc010nxq.1,ENST00000518655,ENSG00000223972,1483
1,uc009viu.3,ENST00000438504,ENSG00000227232,1783
3,uc001aak.3,ENST00000417324,ENSG00000237613,1187
